In [1]:
import numpy as np
import os
import time
from agents import Agent, AlphaFour
import helpers
from collections import namedtuple, deque
from random import choice, sample
import importlib

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras

Using plaidml.keras.backend backend.


In [2]:
Data = namedtuple('Data', 'S P V')

def self_play(agent1, agent2):
    game_board = np.zeros((6,7))
    turn = 0
        
    states = []
    probs = []
    values = []
    
    state = AlphaFour.get_game_state(game_board)
    
    while True:
        states.append(state.copy())  # Add current state

        if state[2].all():
            # Get best move, and probability of all moves from current state
            move,prob = agent1.get_move_with_prob(state)
            state[0] += move
            state[2] = np.zeros((6,7))
        else:
            move,prob = agent2.get_move_with_prob(state)
            state[1] += move
            state[2] = np.ones((6,7))
        
        probs.append(prob) # Store move probabilities from current state
                
        val = helpers.winner(state[0] - state[1])
        if val is not None:
            # Add the final state to our arrays
            states.append(state.copy())
            probs.append(np.zeros((1,7)))
            
            val = -1*abs(val) # If game is over, current player lost unless it's a tie. 
            break
        
        turn += 1
                
    for i in range(len(probs)):
        values.append(val * (-1)**i)
    values = values[::-1]
    
    data = [Data(states[i], probs[i], values[i]) for i in range(len(probs))]
                
    return data

In [3]:
D = self_play(AlphaFour('P1'), AlphaFour('P2'))

/Users/benholmes/Documents/Projects/AlphaFour/agents/alphafour.py:73: RuntimeWarning: invalid value encountered in true_divide
  ucb = np.where(node.N > 0, node.W/node.N + self._EXPLORATION_CONSTANT*node.P/(1+node.N), np.inf)


Winner took 0.000186920166015625
Winner took 0.00010013580322265625
Winner took 8.916854858398438e-05
Winner took 6.604194641113281e-05
Winner took 0.00010180473327636719
Winner took 7.796287536621094e-05
Winner took 8.416175842285156e-05
Winner took 6.29425048828125e-05
Winner took 8.487701416015625e-05
Winner took 7.104873657226562e-05
Winner took 7.891654968261719e-05
Winner took 8.511543273925781e-05
Winner took 0.00012803077697753906
Winner took 9.202957153320312e-05
Winner took 7.891654968261719e-05
Winner took 5.602836608886719e-05
Winner took 6.008148193359375e-05
Winner took 6.723403930664062e-05
Winner took 6.508827209472656e-05
Winner took 5.507469177246094e-05
Winner took 6.103515625e-05
Winner took 6.079673767089844e-05
Winner took 5.602836608886719e-05
Winner took 5.817413330078125e-05
Winner took 6.508827209472656e-05
Winner took 6.914138793945312e-05
Winner took 7.009506225585938e-05
Winner took 9.584426879882812e-05
Winner took 8.0108642578125e-05
Winner took 9.4175338

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
REPLAY_BUFFER_SIZE = 100000  # Number of past steps to store. This is where our training sample is drawn from
SELF_PLAY_BATCH_SIZE = 100 # How many games to play before updating the buffer
TRAINING_SET_SIZE = 1024    # Size of the training set to sample from the replay buffer

replay_buffer = deque(maxlen=REPLAY_BUFFER_SIZE)

# START OF ONE TRAINING LOOP
# ==========================================================================
players = [AlphaFour('Best'), AlphaFour('New')]

total_count = 0

# Generate new self play games.
for ii in range(SELF_PLAY_BATCH_SIZE):
    print(f'\r{ii}', end='')
    i = np.random.randint(2)  # randomize who plays first
    p1 = players[i]
    p2 = players[(i+1)%2]
    
    D = self_play(p1,p2)
    
    for d in D:
        total_count += 1
        replay_buffer.append(d)
        
train_set = sample(replay_buffer, TRAINING_SET_SIZE)

# Train new bot with updated data

# Play matches between new and old. If new wins more than 55%, replace old with New


# ==========================================================================

In [ ]:
state = train_set[123].S
moves = helpers.get_legal_moves(state[0] + state[1])
moves = moves[1:,:,:]
print(moves)

for col in range(7):
    print(moves[:,:,col].sum())

In [ ]:
state = train_set[123].S
moves = helpers.get_legal_moves(state[0] + state[1])
moves = moves[1:]

col_has_move = moves.sum(axis=0).sum(axis=0)

move_idx = 0
for i in range(7):
    if col_has_move[i]:
        new_state = state + np.array([moves[move_idx], np.zeros((6,7)), np.zeros((6,7))])
        move_idx += 1
        print(new_state, '\n\n')
        